# **AI TECH INSTITUTE** · *Intermediate AI & Data Science*
### Week 03 · Notebook 06 — Advanced SQL Analytics
**Instructor:** Amir Charkhi  |  **Goal:** Master the Art of Complex Analytical Queries

> Format: theory → implementation → best practices → real-world application.
>
**Learning Objectives:**
- Master window functions for advanced analytics
- Build complex queries with Common Table Expressions (CTEs)
- Implement recursive queries for hierarchical data
- Perform pivoting and unpivoting operations
- Optimize complex analytical queries


## 🎯 The Analytics Challenge

You're the Senior Data Analyst at an e-commerce company. The CEO wants:
1. **Customer cohort retention analysis**
2. **Product sales rankings with trends**
3. **Employee hierarchy reports**
4. **Time-series analysis with moving averages**

Simple GROUP BY won't cut it. Time for **advanced SQL**!

In [ ]:
# Setup and imports
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display
import warnings
warnings.filterwarnings('ignore')

# Configure displays
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)
sns.set_style('whitegrid')

def show_sql(query, title="SQL Query:"):
    """Pretty print SQL queries"""
    print(f"\n📝 {title}")
    display(Markdown(f"```sql\n{query}\n```"))

def run_query(query, conn, title="Results:"):
    """Execute query and display results"""
    result = pd.read_sql(query, conn)
    print(f"\n📊 {title}")
    display(result)
    return result

print("✅ Environment ready for advanced SQL analytics!")

## 📊 Setting Up Our Analytics Database

We'll create a comprehensive e-commerce database with sales, customers, products, and employees.

In [ ]:
# Create database
conn = sqlite3.connect('analytics.db')
cursor = conn.cursor()

# Drop existing tables
tables = ['sales', 'customers', 'products', 'employees', 'categories']
for table in tables:
    cursor.execute(f"DROP TABLE IF EXISTS {table}")

# Create tables
cursor.execute("""
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    customer_name TEXT,
    signup_date DATE,
    country TEXT,
    customer_segment TEXT
)
""")

cursor.execute("""
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    category TEXT,
    price DECIMAL(10,2),
    launch_date DATE
)
""")

cursor.execute("""
CREATE TABLE sales (
    sale_id INTEGER PRIMARY KEY,
    sale_date DATE,
    customer_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    revenue DECIMAL(10,2),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)
""")

cursor.execute("""
CREATE TABLE employees (
    employee_id INTEGER PRIMARY KEY,
    employee_name TEXT,
    manager_id INTEGER,
    department TEXT,
    salary DECIMAL(10,2),
    hire_date DATE,
    FOREIGN KEY (manager_id) REFERENCES employees(employee_id)
)
""")

cursor.execute("""
CREATE TABLE categories (
    category_id INTEGER PRIMARY KEY,
    category_name TEXT,
    parent_category_id INTEGER,
    FOREIGN KEY (parent_category_id) REFERENCES categories(category_id)
)
""")

conn.commit()
print("✅ Database schema created!")

In [ ]:
# Generate sample data
np.random.seed(42)

# Customers
n_customers = 1000
customers = pd.DataFrame({
    'customer_id': range(1, n_customers + 1),
    'customer_name': [f'Customer_{i}' for i in range(1, n_customers + 1)],
    'signup_date': pd.date_range('2022-01-01', periods=n_customers, freq='6H').date,
    'country': np.random.choice(['USA', 'UK', 'Germany', 'France', 'Japan'], n_customers),
    'customer_segment': np.random.choice(['Premium', 'Standard', 'Basic'], n_customers, p=[0.2, 0.5, 0.3])
})

# Products
n_products = 100
categories = ['Electronics', 'Clothing', 'Books', 'Home', 'Sports']
products = pd.DataFrame({
    'product_id': range(1, n_products + 1),
    'product_name': [f'Product_{i}' for i in range(1, n_products + 1)],
    'category': np.random.choice(categories, n_products),
    'price': np.round(np.random.uniform(10, 500, n_products), 2),
    'launch_date': pd.date_range('2021-01-01', periods=n_products, freq='3D').date
})

# Sales (multiple purchases per customer)
n_sales = 20000
sales_dates = pd.date_range('2022-01-01', '2024-12-31', freq='H')
sales = pd.DataFrame({
    'sale_id': range(1, n_sales + 1),
    'sale_date': np.random.choice(sales_dates, n_sales).date,
    'customer_id': np.random.randint(1, n_customers + 1, n_sales),
    'product_id': np.random.randint(1, n_products + 1, n_sales),
    'quantity': np.random.randint(1, 5, n_sales)
})

# Calculate revenue based on product prices
sales = sales.merge(products[['product_id', 'price']], on='product_id')
sales['revenue'] = sales['quantity'] * sales['price']
sales = sales.drop('price', axis=1)

# Employees with hierarchy
employees = pd.DataFrame({
    'employee_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'employee_name': ['CEO', 'CTO', 'CFO', 'VP_Eng', 'VP_Sales', 'VP_Finance',
                      'Eng_Lead1', 'Eng_Lead2', 'Sales_Lead1', 'Sales_Lead2',
                      'Engineer1', 'Engineer2', 'Engineer3', 'Sales_Rep1', 'Sales_Rep2'],
    'manager_id': [None, 1, 1, 2, 2, 3, 4, 4, 5, 5, 7, 7, 8, 9, 10],
    'department': ['Executive', 'Technology', 'Finance', 'Engineering', 'Sales', 'Finance',
                  'Engineering', 'Engineering', 'Sales', 'Sales',
                  'Engineering', 'Engineering', 'Engineering', 'Sales', 'Sales'],
    'salary': [500000, 350000, 350000, 250000, 250000, 200000,
              180000, 180000, 150000, 150000,
              120000, 120000, 110000, 90000, 85000],
    'hire_date': pd.date_range('2020-01-01', periods=15, freq='2M').date
})

# Category hierarchy
categories_df = pd.DataFrame({
    'category_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'category_name': ['All Products', 'Electronics', 'Clothing', 'Home',
                     'Computers', 'Audio', 'Men', 'Women', 'Kitchen', 'Furniture'],
    'parent_category_id': [None, 1, 1, 1, 2, 2, 3, 3, 4, 4]
})

# Load data into database
customers.to_sql('customers', conn, if_exists='append', index=False)
products.to_sql('products', conn, if_exists='append', index=False)
sales.to_sql('sales', conn, if_exists='append', index=False)
employees.to_sql('employees', conn, if_exists='append', index=False)
categories_df.to_sql('categories', conn, if_exists='append', index=False)

print(f"✅ Loaded data:")
print(f"  - {len(customers):,} customers")
print(f"  - {len(products):,} products")
print(f"  - {len(sales):,} sales transactions")
print(f"  - {len(employees):,} employees")
print(f"  - {len(categories_df):,} categories")

---

## 🪟 Part 1: Window Functions - Analytics Without Grouping

Window functions are SQL's superpower for complex analytics. They calculate across rows while keeping all row details!

### 1.1 Ranking Functions

ROW_NUMBER(), RANK(), DENSE_RANK(), NTILE() - each has its purpose!

In [ ]:
# Compare different ranking functions
ranking_query = """
WITH product_sales AS (
    SELECT 
        p.product_name,
        p.category,
        SUM(s.revenue) as total_revenue
    FROM sales s
    JOIN products p ON s.product_id = p.product_id
    GROUP BY p.product_id, p.product_name, p.category
)
SELECT 
    category,
    product_name,
    total_revenue,
    ROW_NUMBER() OVER (PARTITION BY category ORDER BY total_revenue DESC) as row_num,
    RANK() OVER (PARTITION BY category ORDER BY total_revenue DESC) as rank,
    DENSE_RANK() OVER (PARTITION BY category ORDER BY total_revenue DESC) as dense_rank,
    NTILE(4) OVER (PARTITION BY category ORDER BY total_revenue DESC) as quartile
FROM product_sales
ORDER BY category, total_revenue DESC
LIMIT 15
"""

show_sql(ranking_query, "Ranking Functions Comparison")
ranking_result = run_query(ranking_query, conn, "Different ranking methods:")

print("\n💡 Key Differences:")
print("- ROW_NUMBER: Always unique (1,2,3,4...)")
print("- RANK: Ties get same rank, gaps after (1,2,2,4...)")
print("- DENSE_RANK: Ties get same rank, no gaps (1,2,2,3...)")
print("- NTILE: Divides into equal buckets")

### 1.2 Top-N Per Group Analysis

In [ ]:
# Find top 3 products in each category
top_n_query = """
WITH ranked_products AS (
    SELECT 
        p.category,
        p.product_name,
        SUM(s.revenue) as total_revenue,
        COUNT(DISTINCT s.customer_id) as unique_customers,
        RANK() OVER (
            PARTITION BY p.category 
            ORDER BY SUM(s.revenue) DESC
        ) as revenue_rank
    FROM sales s
    JOIN products p ON s.product_id = p.product_id
    GROUP BY p.product_id, p.product_name, p.category
)
SELECT 
    category,
    product_name,
    total_revenue,
    unique_customers,
    revenue_rank
FROM ranked_products
WHERE revenue_rank <= 3
ORDER BY category, revenue_rank
"""

show_sql(top_n_query, "Top-N Per Group")
top_n_result = run_query(top_n_query, conn, "Top 3 products per category:")

### 1.3 Running Totals and Moving Averages

In [ ]:
# Calculate running totals and moving averages
time_series_query = """
WITH daily_sales AS (
    SELECT 
        sale_date,
        SUM(revenue) as daily_revenue,
        COUNT(DISTINCT customer_id) as daily_customers
    FROM sales
    WHERE sale_date >= '2024-01-01' 
      AND sale_date <= '2024-01-31'
    GROUP BY sale_date
)
SELECT 
    sale_date,
    daily_revenue,
    daily_customers,
    -- Running total
    SUM(daily_revenue) OVER (
        ORDER BY sale_date 
        ROWS UNBOUNDED PRECEDING
    ) as cumulative_revenue,
    -- 7-day moving average
    AVG(daily_revenue) OVER (
        ORDER BY sale_date 
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) as moving_avg_7d,
    -- Month-to-date average
    AVG(daily_revenue) OVER (
        ORDER BY sale_date
        ROWS UNBOUNDED PRECEDING
    ) as mtd_avg_revenue
FROM daily_sales
ORDER BY sale_date
"""

show_sql(time_series_query, "Time Series Analytics")
time_series_result = run_query(time_series_query, conn, "Daily sales with running metrics:")

# Visualize the results
if len(time_series_result) > 0:
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    ax1.plot(pd.to_datetime(time_series_result['sale_date']), 
             time_series_result['daily_revenue'], 
             label='Daily Revenue', marker='o', markersize=3)
    ax1.plot(pd.to_datetime(time_series_result['sale_date']), 
             time_series_result['moving_avg_7d'], 
             label='7-Day Moving Avg', linewidth=2)
    ax1.set_title('Daily Revenue with Moving Average')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    ax2.plot(pd.to_datetime(time_series_result['sale_date']), 
             time_series_result['cumulative_revenue'], 
             label='Cumulative Revenue', color='green', linewidth=2)
    ax2.set_title('Cumulative Revenue (Month-to-Date)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

### 1.4 LAG and LEAD - Time Comparisons

In [ ]:
# Compare with previous period using LAG
lag_lead_query = """
WITH monthly_sales AS (
    SELECT 
        strftime('%Y-%m', sale_date) as month,
        SUM(revenue) as monthly_revenue,
        COUNT(DISTINCT customer_id) as monthly_customers
    FROM sales
    WHERE sale_date >= '2024-01-01'
    GROUP BY strftime('%Y-%m', sale_date)
)
SELECT 
    month,
    monthly_revenue,
    monthly_customers,
    -- Previous month
    LAG(monthly_revenue, 1) OVER (ORDER BY month) as prev_month_revenue,
    -- Month-over-month change
    monthly_revenue - LAG(monthly_revenue, 1) OVER (ORDER BY month) as mom_change,
    -- Month-over-month percentage
    ROUND(
        (monthly_revenue - LAG(monthly_revenue, 1) OVER (ORDER BY month)) * 100.0 / 
        NULLIF(LAG(monthly_revenue, 1) OVER (ORDER BY month), 0), 
        2
    ) as mom_change_pct,
    -- Next month (for forecasting)
    LEAD(monthly_revenue, 1) OVER (ORDER BY month) as next_month_revenue
FROM monthly_sales
ORDER BY month
LIMIT 12
"""

show_sql(lag_lead_query, "LAG/LEAD for Time Comparisons")
lag_lead_result = run_query(lag_lead_query, conn, "Month-over-month analysis:")

---

## 🔄 Part 2: Common Table Expressions (CTEs)

CTEs make complex queries readable and maintainable.

### 2.1 Multi-Step Analysis with CTEs

In [ ]:
# Complex cohort retention analysis using multiple CTEs
cohort_query = """
WITH 
-- Step 1: Identify first purchase date for each customer
customer_cohorts AS (
    SELECT 
        customer_id,
        MIN(sale_date) as first_purchase_date,
        strftime('%Y-%m', MIN(sale_date)) as cohort_month
    FROM sales
    GROUP BY customer_id
),
-- Step 2: Calculate months since first purchase
customer_purchases AS (
    SELECT 
        s.customer_id,
        c.cohort_month,
        strftime('%Y-%m', s.sale_date) as purchase_month,
        (strftime('%Y', s.sale_date) - strftime('%Y', c.first_purchase_date)) * 12 +
        (strftime('%m', s.sale_date) - strftime('%m', c.first_purchase_date)) as months_since_first
    FROM sales s
    JOIN customer_cohorts c ON s.customer_id = c.customer_id
),
-- Step 3: Count unique customers per cohort
cohort_sizes AS (
    SELECT 
        cohort_month,
        COUNT(DISTINCT customer_id) as cohort_size
    FROM customer_cohorts
    GROUP BY cohort_month
),
-- Step 4: Calculate retention
retention_data AS (
    SELECT 
        cp.cohort_month,
        cp.months_since_first,
        COUNT(DISTINCT cp.customer_id) as retained_customers
    FROM customer_purchases cp
    GROUP BY cp.cohort_month, cp.months_since_first
)
-- Final: Calculate retention rates
SELECT 
    r.cohort_month,
    r.months_since_first,
    r.retained_customers,
    c.cohort_size,
    ROUND(r.retained_customers * 100.0 / c.cohort_size, 2) as retention_rate
FROM retention_data r
JOIN cohort_sizes c ON r.cohort_month = c.cohort_month
WHERE r.cohort_month >= '2022-01' 
  AND r.cohort_month <= '2022-06'
  AND r.months_since_first <= 12
ORDER BY r.cohort_month, r.months_since_first
"""

show_sql(cohort_query, "Complex Cohort Analysis with CTEs")
cohort_result = run_query(cohort_query, conn, "Cohort retention analysis:")

# Visualize cohort retention
if len(cohort_result) > 0:
    pivot_cohort = cohort_result.pivot(index='cohort_month', 
                                       columns='months_since_first', 
                                       values='retention_rate')
    
    plt.figure(figsize=(14, 6))
    sns.heatmap(pivot_cohort, annot=True, fmt='.1f', cmap='YlOrRd', 
                cbar_kws={'label': 'Retention Rate (%)'})
    plt.title('Cohort Retention Analysis')
    plt.xlabel('Months Since First Purchase')
    plt.ylabel('Cohort Month')
    plt.show()

---

## 🌳 Part 3: Recursive CTEs - Hierarchical Data

Recursive CTEs are perfect for organizational charts and category trees.

In [ ]:
# Employee hierarchy using recursive CTE
hierarchy_query = """
WITH RECURSIVE employee_hierarchy AS (
    -- Anchor: Start with the CEO
    SELECT 
        employee_id,
        employee_name,
        manager_id,
        department,
        salary,
        0 as level,
        employee_name as path
    FROM employees
    WHERE manager_id IS NULL
    
    UNION ALL
    
    -- Recursive: Add subordinates
    SELECT 
        e.employee_id,
        e.employee_name,
        e.manager_id,
        e.department,
        e.salary,
        h.level + 1,
        h.path || ' > ' || e.employee_name
    FROM employees e
    INNER JOIN employee_hierarchy h ON e.manager_id = h.employee_id
)
SELECT 
    level,
    substr('    ', 1, level * 2) || employee_name as org_chart,
    department,
    salary,
    path
FROM employee_hierarchy
ORDER BY path
"""

show_sql(hierarchy_query, "Recursive CTE for Employee Hierarchy")
hierarchy_result = run_query(hierarchy_query, conn, "Organization Chart:")

---

## 🔄 Part 4: PIVOT Operations

Transform rows to columns for different analytical views.

In [ ]:
# Manual PIVOT using CASE statements
pivot_query = """
WITH monthly_category_sales AS (
    SELECT 
        strftime('%Y-%m', s.sale_date) as month,
        p.category,
        SUM(s.revenue) as revenue
    FROM sales s
    JOIN products p ON s.product_id = p.product_id
    WHERE s.sale_date >= '2024-01-01' 
      AND s.sale_date <= '2024-06-30'
    GROUP BY strftime('%Y-%m', s.sale_date), p.category
)
SELECT 
    month,
    SUM(CASE WHEN category = 'Electronics' THEN revenue ELSE 0 END) as Electronics,
    SUM(CASE WHEN category = 'Clothing' THEN revenue ELSE 0 END) as Clothing,
    SUM(CASE WHEN category = 'Books' THEN revenue ELSE 0 END) as Books,
    SUM(CASE WHEN category = 'Home' THEN revenue ELSE 0 END) as Home,
    SUM(CASE WHEN category = 'Sports' THEN revenue ELSE 0 END) as Sports,
    SUM(revenue) as Total
FROM monthly_category_sales
GROUP BY month
ORDER BY month
"""

show_sql(pivot_query, "PIVOT: Monthly Sales by Category")
pivot_result = run_query(pivot_query, conn, "Pivoted sales data:")

# Visualize pivoted data
if len(pivot_result) > 0:
    pivot_result.set_index('month')[['Electronics', 'Clothing', 'Books', 'Home', 'Sports']].plot(
        kind='bar', stacked=True, figsize=(12, 6), colormap='Set3'
    )
    plt.title('Monthly Sales by Category (Stacked)')
    plt.xlabel('Month')
    plt.ylabel('Revenue')
    plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

---

## 🚀 Part 5: Advanced Analytical Patterns

Let's combine everything for complex real-world analytics!

### 5.1 RFM Analysis (Recency, Frequency, Monetary)

In [ ]:
# RFM Analysis for Customer Segmentation
rfm_query = """
WITH customer_rfm AS (
    SELECT 
        c.customer_id,
        c.customer_name,
        c.customer_segment,
        -- Recency: Days since last purchase
        JULIANDAY('2024-12-31') - JULIANDAY(MAX(s.sale_date)) as recency,
        -- Frequency: Number of purchases
        COUNT(DISTINCT s.sale_id) as frequency,
        -- Monetary: Total spend
        COALESCE(SUM(s.revenue), 0) as monetary
    FROM customers c
    LEFT JOIN sales s ON c.customer_id = s.customer_id
    GROUP BY c.customer_id, c.customer_name, c.customer_segment
),
rfm_scores AS (
    SELECT 
        *,
        -- Score each dimension 1-5 (5 is best)
        NTILE(5) OVER (ORDER BY recency DESC) as r_score,
        NTILE(5) OVER (ORDER BY frequency) as f_score,
        NTILE(5) OVER (ORDER BY monetary) as m_score
    FROM customer_rfm
    WHERE frequency > 0
)
SELECT 
    customer_id,
    customer_name,
    customer_segment,
    ROUND(recency, 0) as recency_days,
    frequency,
    ROUND(monetary, 2) as monetary,
    r_score,
    f_score,
    m_score,
    r_score || f_score || m_score as rfm_combined,
    CASE 
        WHEN r_score >= 4 AND f_score >= 4 AND m_score >= 4 THEN 'Champions'
        WHEN r_score >= 3 AND f_score >= 3 AND m_score >= 4 THEN 'Loyal Customers'
        WHEN r_score >= 3 AND f_score <= 2 AND m_score >= 3 THEN 'Potential Loyalists'
        WHEN r_score >= 4 AND f_score <= 2 AND m_score <= 2 THEN 'New Customers'
        WHEN r_score <= 2 AND f_score >= 3 AND m_score >= 3 THEN 'At Risk'
        WHEN r_score <= 2 AND f_score <= 2 AND m_score >= 3 THEN 'Cant Lose Them'
        ELSE 'Others'
    END as rfm_segment
FROM rfm_scores
ORDER BY monetary DESC
LIMIT 20
"""

show_sql(rfm_query, "RFM Analysis for Customer Segmentation")
rfm_result = run_query(rfm_query, conn, "RFM Customer Segmentation:")

---

## 🎯 Practice Exercises

Now it's your turn! Try these advanced SQL challenges.

### Exercise 1: Funnel Analysis

Create a conversion funnel showing customer progression.

In [ ]:
# TODO: Create a funnel analysis
# Stages: All customers -> Made 1 purchase -> Made 2+ purchases -> Made 5+ purchases

print("Exercise: Create a funnel analysis")
print("Hint: Use CTEs to calculate each stage")
print("Hint: Calculate conversion rates between stages")

### Exercise 2: Pareto Analysis (80/20 Rule)

Find what percentage of customers generate 80% of revenue.

In [ ]:
# TODO: Implement Pareto analysis
print("Exercise: Implement Pareto analysis")
print("Hint: Use cumulative sum window function")
print("Hint: Find where cumulative revenue reaches 80%")

---

## 🎓 Key Takeaways

1. **Window Functions**:
   - Calculate across rows without grouping
   - Essential for rankings, running totals, comparisons
   - PARTITION BY creates windows, ORDER BY defines sequence

2. **CTEs (Common Table Expressions)**:
   - Break complex queries into readable steps
   - Reusable within the same query
   - Better than nested subqueries for clarity

3. **Recursive CTEs**:
   - Perfect for hierarchical data
   - Self-referencing for tree traversal
   - Use anchor + recursive members

4. **PIVOT Operations**:
   - Transform rows to columns
   - Use CASE statements for manual pivot
   - Essential for reporting formats

5. **Advanced Patterns**:
   - RFM for customer segmentation
   - Cohort analysis for retention
   - Time series decomposition

---

## 🚀 Next Steps

In the next notebook:
- Query optimization and performance tuning
- Cloud data warehouse features
- Production SQL best practices

Remember: **Advanced SQL transforms data into insights!** 📊

In [ ]:
# Cleanup
conn.close()
print("✅ Database connection closed. Excellent work mastering advanced SQL!")